In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras import regularizers

2024-02-10 00:13:05.766987: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-10 00:13:05.793004: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-10 00:13:05.793037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-10 00:13:05.793866: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-10 00:13:05.798645: I tensorflow/core/platform/cpu_feature_guar

In [2]:
df1 = pd.read_csv("book1.csv")
df2 = pd.read_csv("book.csv")

df = pd.concat([df1, df2])

In [3]:
df["label"].value_counts()

label
medical        20
legal          20
financial      20
educational    20
business       20
news           20
technical      20
creative       20
scientific     20
government     20
Name: count, dtype: int64

In [4]:
def preprocess(text):
    text = re.sub("[_\n]", " ", text)
    return re.sub(r" +", "", text)


df["text"] = df["text"].apply(preprocess)

In [5]:
bert_preprocess = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
)
bert_encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"
)

2024-02-10 00:13:07.542915: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 00:13:07.634063: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 00:13:07.634217: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

In [7]:
df

,label,text
0,6,MedicalHistoryFormPatientInformation:Name:Edwa...
1,6,PhysicalExaminationReportPatientInformation:Na...
2,6,LaboratoryTestResults(BloodTest)PatientInforma...
3,6,RadiologyReport(X-ray)PatientInformation:Name:...
4,6,PrescriptionRecordPatientInformation:Name:Rohi...
...,...,...
95,4,PublicPolicyReportTitle:NationalClimateChangeP...
96,4,EnvironmentalImpactStatement(EIS)Title:Propose...
97,4,PublicHealthAdvisoryTitle:CentersforDiseaseCon...
98,4,GovernmentAuditReportTitle:InternalRevenueServ...


In [8]:
df = df.sample(frac=1, random_state=30)
train_data, temp_data = train_test_split(
    df, test_size=0.2, stratify=df["label"], random_state=30
)
dev_data, test_data = train_test_split(
    temp_data, test_size=0.5, stratify=temp_data["label"], random_state=30
)

In [9]:
import numpy as np

X_train = np.array((train_data["text"].tolist()))
y_train = np.array(to_categorical(train_data["label"].tolist(), num_classes=10))
X_dev = np.array((dev_data["text"].tolist()))
y_dev = np.array(to_categorical(dev_data["label"].tolist(), num_classes=10))
X_test = np.array((test_data["text"].tolist()))
y_test = np.array(to_categorical(test_data["label"].tolist(), num_classes=10))

In [10]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [11]:
lambda_reg = 0.01

In [12]:
dropout_layer1 = tf.keras.layers.Dropout(0.1)(outputs["pooled_output"])
dense_layer1 = tf.keras.layers.Dense(
    512, activation="relu", kernel_regularizer=regularizers.l2(lambda_reg)
)(dropout_layer1)
dropout_layer2 = tf.keras.layers.Dropout(0.1)(dense_layer1)
dense_layer2 = tf.keras.layers.Dense(
    512, activation="relu", kernel_regularizer=regularizers.l2(lambda_reg)
)(dropout_layer2)
output = tf.keras.layers.Dense(10, activation="softmax", name="output")(dense_layer2)
model = tf.keras.Model(inputs=[text_input], outputs=output)

In [13]:
batch_size = 32

In [14]:
STEPS_PER_EPOCH = len(X_train) // batch_size
#
custom_lr = PolynomialDecay(
    initial_learning_rate=0.01, decay_steps=STEPS_PER_EPOCH, end_learning_rate=0.0001
)
# # Now you can use STEPS_PER_EPOCH in your learning rate schedule
# custom_lr = tf.keras.optimizers.schedules.InverseTimeDecay(
#     0.001, decay_steps=STEPS_PER_EPOCH * 1000, decay_rate=1, staircase=False
# )

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=custom_lr),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

2024-02-10 00:13:13.012515: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [16]:
early_stopping = EarlyStopping(monitor="val_loss", patience=3)
checkpoint = ModelCheckpoint(
    "best_model.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode="max"
)
tensorboard = TensorBoard(log_dir="./logs")

In [17]:
model.evaluate(X_train, y_train)

5/5 [==============================] - 3s 300ms/step - loss: 13.6083 - accuracy: 0.1000


[13.608274459838867, 0.10000000149011612]

In [18]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 1s 693ms/step - loss: 13.5993 - accuracy: 0.1000


[13.599302291870117, 0.10000000149011612]

In [19]:
model.evaluate(X_dev, y_dev)

1/1 [==============================] - 0s 198ms/step - loss: 13.6241 - accuracy: 0.1000


[13.6240873336792, 0.10000000149011612]

In [20]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_dev, y_dev),
    epochs=300,
    batch_size=batch_size,
    callbacks=[early_stopping],
)

Epoch 1/300


2024-02-10 00:13:20.966453: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f372e37afb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-10 00:13:20.966480: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-02-10 00:13:20.981221: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-10 00:13:20.997926: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1707504201.056906   53600 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5/5 [==============================] - 6s 351ms/step - loss: 18.0641 - accuracy: 0.0625 - val_loss: 11.3496 - val_accuracy: 0.1500
Epoch 2/300
5/5 [==============================] - 2s 332ms/step - loss: 11.3203 - accuracy: 0.1312 - val_loss: 11.0533 - val_accuracy: 0.1500
Epoch 3/300
5/5 [==============================] - 2s 331ms/step - loss: 11.0054 - accuracy: 0.1625 - val_loss: 10.6710 - val_accuracy: 0.2000
Epoch 4/300
5/5 [==============================] - 2s 339ms/step - loss: 10.5807 - accuracy: 0.1312 - val_loss: 10.3060 - val_accuracy: 0.2000
Epoch 5/300
5/5 [==============================] - 2s 340ms/step - loss: 10.2629 - accuracy: 0.1500 - val_loss: 9.9575 - val_accuracy: 0.1500
Epoch 6/300
5/5 [==============================] - 2s 328ms/step - loss: 9.8890 - accuracy: 0.1187 - val_loss: 9.6456 - val_accuracy: 0.1500
Epoch 7/300
5/5 [==============================] - 2s 326ms/step - loss: 9.5699 - accuracy: 0.1813 - val_loss: 9.3691 - val_accuracy: 0.1500
Epoch 8/300
5/5 

In [21]:
model.save("pdf_classifier")

INFO:tensorflow:Assets written to: pdf_classifier/assets


INFO:tensorflow:Assets written to: pdf_classifier/assets


In [22]:
model.evaluate(X_test, y_test)
y = model.predict(X_test)
predicted_indices = np.argmax(y, axis=-1)
original_labels = label_encoder.inverse_transform(predicted_indices)

1/1 [==============================] - 0s 472ms/step


In [23]:
print(original_labels)

['scientific' 'medical' 'medical' 'government' 'scientific' 'creative'
 'scientific' 'business' 'business' 'creative' 'business' 'educational'
 'technical' 'business' 'financial' 'government' 'government' 'financial'
 'financial' 'creative']


In [24]:
import pickle

# Save the label encoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)